# Linear Models

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [20]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_blobs
from addutils import css_notebook
from sklearn import metrics
css_notebook()

In [13]:
import bokeh.plotting as bk
from bokeh import palettes
bk.output_notebook()

Loading BokehJS ...

In [43]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

## 1 Introduction

What is a linear model? 
A linear regression model assumes that the regression function $E(Y |X)$ is
linear in the inputs $X_1 , \ldots , X_p$ . Linear models were largely developed in
the precomputer age of statistics, but even in today’s computer era there
are still good reasons to study and use them. They are simple and often
provide an adequate and interpretable description of how the inputs affect
the output. For prediction purposes they can sometimes outperform fancier
nonlinear models, especially in situations with small numbers of training
cases, low signal-to-noise ratio or sparse data. Finally, linear methods can be
applied to transformations of the inputs and this considerably expands their
scope.

---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.